In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from pyspark.sql.functions import to_date

spark.sql("CREATE SCHEMA IF NOT EXISTS retail.customer_data")

customer_schema = StructType([
    StructField("customer_id", IntegerType(), False),
    StructField("name", StringType(), True),
    StructField("region", StringType(), True),
    StructField("status", StringType(), True),
    StructField("signup_date", DateType(), True)
])

customer_data = [
    (1, "Alice Johnson", "West", "Active", "2023-01-15"),
    (2, "Bob Smith", "East", "Inactive", "2023-02-20"),
    (3, "Charlie Lee", "West", "Active", "2023-03-10")
]

raw_df = spark.createDataFrame(customer_data, schema=["customer_id","name","region","status","signup_date"])
customers_df = raw_df.withColumn("signup_date", to_date("signup_date", "yyyy-MM-dd"))

customers_df.write.format("delta").mode("overwrite").saveAsTable("retail.customer_data.customers")

result_df = spark.sql("""
  SELECT customer_id, name, region
  FROM retail.customer_data.customers
  WHERE status = 'Active' AND region = 'West'
""")

display(result_df)


customer_id,name,region
1,Alice Johnson,West
3,Charlie Lee,West


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from delta.tables import DeltaTable

spark = SparkSession.builder.appName("InventoryUpdates").getOrCreate()

spark.sql("CREATE SCHEMA IF NOT EXISTS ecommerce.inventory")

inventory_schema = StructType([
    StructField("product_id", IntegerType(), False),
    StructField("name", StringType(), True),
    StructField("stock", IntegerType(), True),
    StructField("warehouse", StringType(), True)
])

initial_data = [
    (101, "Laptop", 50, "A"),
    (102, "Smartphone", 100, "B")
]
initial_df = spark.createDataFrame(initial_data, schema=inventory_schema)
initial_df.write.format("delta").mode("overwrite").saveAsTable("ecommerce.inventory.products")

update_data = [
    (101, "Laptop", 45, "A"),
    (103, "Tablet", 30, "A")
]
update_df = spark.createDataFrame(update_data, schema=inventory_schema)

delta_table = DeltaTable.forName(spark, "ecommerce.inventory.products")
delta_table.alias("target").merge(
    update_df.alias("source"),
    "target.product_id = source.product_id"
).whenMatchedUpdate(
    set={"stock": "source.stock"}
).whenNotMatchedInsert(
    values={
        "product_id": "source.product_id",
        "name": "source.name",
        "stock": "source.stock",
        "warehouse": "source.warehouse"
    }
).execute()

final_df = spark.sql("SELECT * FROM ecommerce.inventory.products ORDER BY product_id")
display(final_df)


product_id,name,stock,warehouse
101,Laptop,45,A
102,Smartphone,100,B
103,Tablet,30,A


In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from delta.tables import DeltaTable

spark.sql("CREATE SCHEMA IF NOT EXISTS finance.sales")

sales_schema = StructType([
    StructField("sale_id", IntegerType(), False),
    StructField("product", StringType(), True),
    StructField("amount", IntegerType(), True),
    StructField("quarter", StringType(), True)
])

initial_data = [
    (1, "Stocks", 10000, "Q1-2023"),
    (2, "Bonds", 15000, "Q1-2023")
]
initial_df = spark.createDataFrame(initial_data, schema=sales_schema)
initial_df.write.format("delta").mode("overwrite").saveAsTable("finance.sales.quarterly_sales")

update_data = [(1, "Stocks", 12000, "Q1-2023")]
update_df = spark.createDataFrame(update_data, schema=sales_schema)

delta_table = DeltaTable.forName(spark, "finance.sales.quarterly_sales")
delta_table.alias("target").merge(
    update_df.alias("source"),
    "target.sale_id = source.sale_id"
).whenMatchedUpdate(set={"amount": "source.amount"}).execute()

previous_version = delta_table.history().select("version").collect()[1][0]
spark.sql(f"""
    SELECT SUM(amount) AS total_amount
    FROM finance.sales.quarterly_sales VERSION AS OF {previous_version}
""").show()

spark.sql("""
    SELECT SUM(amount) AS total_amount
    FROM finance.sales.quarterly_sales
""").show()


+------------+
|total_amount|
+------------+
|       25000|
+------------+

+------------+
|total_amount|
+------------+
|       27000|
+------------+

